In [28]:
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 

# read csv file using pandas
df1 = pd.read_csv('final_Movies_data0.csv')
df2 = pd.read_csv('Translated_telugu_dataset .csv')

df1.rename(columns = {'title/الفيلم':'Title','cast/تمثيل':'Cast','genres/النوع':'Genre','Year/السنة':'Year','original_language/اللغة':'Language',},inplace = True)
df2.rename(columns = {'సినిమా పేరు':'Title', 'దర్శకుడు':'Director', 'నటులు':'Cast', 'శైలి':'Genre', 'తేదీ':'Date','సంవత్సరం':'Year','నెల':'Month','భాష':'Language','Soup/సూప్':'Soup' }, inplace = True)
df2['Language'] = 'te'

# remove brackets
df1['Cast'] = df1['Cast'].str.strip('[]')
df1['Genre'] = df1['Genre'].str.strip('[]')

# remove quotes
df1["Cast"] = df1["Cast"].str.replace(r"^'|,'$", "").str.replace("'", '')
df1["Genre"] = df1["Genre"].str.replace(r"^'|,'$", "").str.replace("'", '')

# inorder to add year into soup we need to conevrt into string formte
df1['Year']= df1['Year'].apply(str)

df1 = df1[~df1.Title.isin(df1[df1.duplicated(subset=['Title'])].Title.unique())].reset_index(drop=True)
df2 = df2[~df2.Title.isin(df2[df2.duplicated(subset=['Title'])].Title.unique())].reset_index(drop=True)

data_ref = pd.concat([df1[['Title', 'Cast', 'Year', 'Genre', 'Language']], df2[['Title', 'Cast', 'Year', 'Genre', 'Language']]]).dropna()

df1['Cast'] = df1['Cast'].str.replace(" ", '')

# create the new soup
df1['Soup'] = df1['Cast']+ '  ' + df1['Genre']+ '  ' + '  ' + df1['Language']

# Lower casing
df1['Soup'] = df1['Soup'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

# remove comma from soup
# df1['Soup'] = df1['Soup'].str.replace(',', '')
df2['Soup'] = df2['Cast']+ '  ' + df2['Genre']+ '  ' + '  ' + df2['Language']

data = pd.concat([df1[['Soup']], df2[['Soup']]]).dropna()

count = TfidfVectorizer(stop_words='english')
count_matrix1 = count.fit_transform(data['Soup'])

/var/folders/ky/f7y74l5j3qg23vm3ysrdy97m0000gn/T/ipykernel_50855/1029292480.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["Cast"] = df1["Cast"].str.replace(r"^'|,'$", "").str.replace("'", '')
/var/folders/ky/f7y74l5j3qg23vm3ysrdy97m0000gn/T/ipykernel_50855/1029292480.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["Genre"] = df1["Genre"].str.replace(r"^'|,'$", "").str.replace("'", '')


In [63]:
searchTerms = 'عادلإمام'
count_matrix2 = count.transform([searchTerms])
cosine_sim2 = cosine_similarity(count_matrix1, count_matrix2) #getting a similarity matrix
print(sorted(list(enumerate(cosine_sim2)), key=lambda x: x[1], reverse=True)[:10])
idx = [i[0] for i in sorted(list(enumerate(cosine_sim2)), key=lambda x: x[1], reverse=True)][:5]
[data_ref.iloc[i]['Title']+' ('+data_ref.iloc[i]['Year']+') | ('+data_ref.iloc[i]['Genre']+') | '+data_ref.iloc[i]['Cast'] for i in idx]


[(10585, array([0.67936174])), (10611, array([0.67936174])), (10644, array([0.67936174])), (10676, array([0.67936174])), (10735, array([0.67936174])), (10760, array([0.67936174])), (10803, array([0.67936174])), (10837, array([0.67936174])), (10189, array([0.27288797])), (10341, array([0.26124799]))]


['أمير الظلام (2002) | (دراما,  اثارة) | عادل إمام',
 'التجربة الدنماركية (2003) | (دراما,  اثارة) | عادل إمام',
 'عريس من جهة أمنية (2004) | (دراما,  اثارة) | عادل إمام',
 'عمارة يعقوبيان (2006) | (دراما,  اثارة) | عادل إمام',
 'مرجان أحمد مرجان (2007) | (دراما,  اثارة) | عادل إمام']

In [61]:
[i for i in count.vocabulary_.keys() if 'عادلإمام' in i]

['عادلإمام']

In [58]:
data_ref

Title  \
0                       Toy Story   
1                         Jumanji   
2                Grumpier Old Men   
3               Waiting to Exhale   
4     Father of the Bride Part II   
...                           ...   
3343                    ఉందా లేదా   
3344        మిడిల్ క్లాసు అబ్బాయి   
3345                          హలో   
3346                   ఒక్క క్షణం   
3347                     2 దేశాలు   

                                                   Cast  Year  \
0     Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...  1995   
1     Robin Williams, Jonathan Hyde, Kirsten Dunst, ...  1995   
2     Walter Matthau, Jack Lemmon, Ann-Margret, Soph...  1995   
3     Whitney Houston, Angela Bassett, Loretta Devin...  1995   
4     Steve Martin, Diane Keaton, Martin Short, Kimb...  1995   
...                                                 ...   ...   
3343                                    రామకృష్ణ, అంకిత  2017   
3344                  నాని, సాయి పల్లవి, Bhumika Chawla  2017   
3345  అఖిల్ అక్కినేని, కళ్యాణి ప్రియదర్శన్, జగపతి బా...  2017   
3346  అల్లు శిరీష్, సురభి, సీరత్ కపూర్, శ్రీనివాస్ అ...  2017   
3347                 సునీల్, మనీషా రాజ్, సంజాజన్, నరేష్  2017   

                           Genre Language  
0      Animation, Comedy, Family       en  
1     Adventure, Fantasy, Family       en  
2                Romance, Comedy       en  
3         Comedy, Drama, Romance       en  
4                         Comedy       en  
...                          ...      ...  
3343                   థ్రిల్లర్       te  
3344            రొమాంటిక్ డ్రామా       te  
3345                        చర్య       te  
3346    సైన్స్ ఫిక్షన్ థ్రిల్లర్       te  
3347            రొమాంటిక్ కామెడీ       te  

[14119 rows x 5 columns]